## get measurement dataset

In [ ]:
import pickle

probe_atlas_file = "../one_measurements/anchors_probes_dataset/all_atlas_probes.pickle"

with open(probe_atlas_file, "rb") as f:
    probes_atlas = pickle.load(f)

anchors = {}
for probe, probe_description in probes_atlas.items():
    if probe_description['is_anchor']:
        anchors[probe] = probe_description

In [ ]:
from random import randint

NB_ANCHOR = 5
target_anchors_index = [randint(0, len(anchors)) for _ in range(0, NB_ANCHOR)]

# get target dataset
target_anchors = {}
anchors_ip = list(anchors.keys())
for index in target_anchors_index:
    target_anchors[anchors_ip[index]] = anchors[anchors_ip[index]]

# get vantage points dataset
vp_anchors = {}
for probe, probe_description in anchors.items():
    if not probe in target_anchors:
        vp_anchors[probe] = probe_description

In [ ]:
print("targets")
for index, (probe, probe_description) in enumerate(target_anchors.items()):
    print(f"target_ip: {probe} : {probe_description}")
print()
print("vps:")
for index, (probe, probe_description) in enumerate(vp_anchors.items()):
    print(f"target_ip: {probe} : {probe_description}")


anchor_file = "../one_measurements/anchors_probes_dataset/anchors.pickle"

with open(anchor_file, "wb") as f:
    pickle.dump(target_anchors, f)

## from ip target list select probes

In [ ]:
import pickle

from utils.helpers import distance

anchor_file = "../one_measurements/anchors_probes_dataset/anchors.pickle"

with open(anchor_file, "rb") as f:
    target_anchors = pickle.load(f)


def get_closest_vp(target_ip: str, target_description: dict, vp_anchors: dict, nb_vp: int):

    distance_probes_target = []
    for vp_ip, vp_description in vp_anchors.items():

        p_lat = vp_description["latitude"]
        p_lon = vp_description["longitude"]

        t_lat = target_description["latitude"]
        t_lon = target_description["longitude"]

        d = distance(p_lat, t_lat, p_lon, t_lon)
        distance_probes_target.append((vp_ip, d))

    distance_probes_target = sorted(distance_probes_target, key=lambda d: d[1])
    closest_vp = [vp_ip for vp_ip, _ in distance_probes_target[:nb_vp]]

    # filter out the target ip
    for vp_ip in closest_vp:
        if target_ip in closest_vp:
            raise RuntimeError(
                "the two datasets should not have IP addresses in common")

    return closest_vp

In [ ]:
NB_VP = 15
measurements = {}
for target_ip, target_description in target_anchors.items():

    # retrieve the ten closest vp for each targets
    measurements[target_ip] = get_closest_vp(
        target_ip, target_description, vp_anchors, NB_VP)

    print(f"traget ip: {target_ip}: {target_description}")
    for vp_ip in measurements[target_ip]:
        print(f"probe ip {vp_ip}: {vp_anchors[vp_ip]}")
    print()

## start measurement

In [ ]:
import time

from one_measurements.src.atlas_probing import RIPEAtlas

'''RIPE_ACCOUNT = "timur.friedman@sorbonne-universite.fr"
RIPE_KEY = "b3d3d4fc-724e-4505-befe-1ad16a70dc87"
NB_PACKETS = 5

driver = RIPEAtlas(RIPE_ACCOUNT, RIPE_KEY)

measurement_ids = []
dry_run = False
for target_ip in measurements:
    print(f"starting measurement for target {target_ip}")
    print(f"with {len(measurements[target_ip])} vp: {[vp_ip for vp_ip in measurements[target_ip]]}")

    # get anchor ids
    vp_ids = [
        vp_anchors[vp_ip]['id']
        for vp_ip in measurements[target_ip]
    ]
    # run measurement
    if dry_run:
        measurement_id = None, None
    else:
        measurement_id = driver.ping(target_ip, vp_ids, NB_PACKETS)
    
    if measurement_id:
        measurement_ids.append(measurement_id)

        print(f"measurement done, measurement id: {measurement_id}")
        print()
    else:
        print("two many concurrent measurements")
        time.sleep(4)'''

In [ ]:
# print(measurement_ids)

# retreive measurements results

import pickle
import requests

from collections import defaultdict
from pathlib import Path

from utils.helpers import rtt_to_km, distance

measurement_file = Path(".") / "../anchors_probes_dataset/probes.pickle"
probe_atlas_file = "../anchors_probes_dataset/all_atlas_probes.pickle"

with open(probe_atlas_file, "rb") as f:
    probes_atlas = pickle.load(f)

anchors = {}
for probe, probe_description in probes_atlas.items():
    if probe_description['is_anchor']:
        anchors[probe] = probe_description


# get measurement results
with open(measurement_file, "rb") as f:
    measurement_results = pickle.load(f)

geoloc_results = defaultdict(dict)


def get_from_atlas(url):
    """request atlas api"""
    response = requests.get(url).json()
    for result in response:
        yield result


key = "b3d3d4fc-724e-4505-befe-1ad16a70dc87"
for measurement_id in measurement_ids:

    url = f"https://atlas.ripe.net/api/v2/measurements/{measurement_id}/results/?key={key}"

    results = get_from_atlas(url)

    for result in results:
        print(result)
        # parse results and calculate geoloc
        if result.get('result') is not None:
            try:
                target_ip = result["dst_name"]
                vp_ip = result['from']
                min_rtt = min([list(rtt.values())[0]
                              for rtt in result['result']])

                # both vp and target coordinates
                vp_lat = anchors[vp_ip]['latitude']
                vp_lon = anchors[vp_ip]['longitude']
                target_lat = anchors[target_ip]["latitude"]
                target_lon = anchors[target_ip]["longitude"]

                # save measurement results for each pair target <-> vp
                geoloc_results[target_ip][vp_ip] = {}
                geoloc_results[target_ip][vp_ip]['rtt'] = min_rtt
                geoloc_results[target_ip][vp_ip]["latitude"] = vp_lat
                geoloc_results[target_ip][vp_ip]["longitude"] = vp_lon

                # compute true distance between vp and target
                geoloc_results[target_ip][vp_ip]["calculated_d"] = distance(
                    vp_lat,
                    target_lat,
                    vp_lon,
                    target_lon
                )

                # compute measured distance based on rtt
                geoloc_results[target_ip][vp_ip]["measured_d"] = rtt_to_km(
                    min_rtt)

                # debugging
                # print(
                #     f"target: {result['dst_addr']}: {anchors[target_ip]} \n \
                #     anchor: {vp_ip}: {anchors[vp_ip]} \n \
                #     mean rtt: {geoloc_results[target_ip][vp_ip]} \n \
                #     distance (calculated): {geoloc_results[target_ip][vp_ip]['calculated_d']} \n \
                #     distance (measured)  : {geoloc_results[target_ip][vp_ip]['measured_d']}"
                # )
            except TypeError:
                continue
        else:
            print(f"no results: {result}")

## Dummy

In [ ]:
import pickle

from timeit import default_timer as timer

from one_measurements.src.atlas_probing import RIPEAtlas

import uuid
tag = str(uuid.uuid4())

# load anchors
anchor_file = "../one_measurements/anchors_probes_dataset/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

driver = RIPEAtlas(ripe_credentials['username'], ripe_credentials["key"])

start = timer()
nb_targets = 4
for i, target_addr in enumerate(anchors):
    if i >= nb_targets:
        break

    vp_ids = [anchors[vp_addr]['id']
              for vp_addr in anchors if vp_addr != target_addr]

    measurement_id = driver.ping(target_addr, vp_ids, tag)

end = timer()

print(f"number of targets: {nb_targets} : {end - start}")

In [ ]:
import pickle

from timeit import default_timer as timer
from multiprocessing.pool import Pool

from one_measurements.src.atlas_probing import RIPEAtlas

import uuid
tag = str(uuid.uuid4())

# load anchors
anchor_file = "../one_measurements/anchors_probes_dataset/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

driver = RIPEAtlas(ripe_credentials['username'], ripe_credentials["key"])

workers = Pool(processes=10)

start = timer()
all_measurements = []
nb_targets = 4
for i, target_addr in enumerate(anchors):
    if i >= nb_targets:
        break

    vp_ids = [anchors[vp_addr]['id']
              for vp_addr in anchors if vp_addr != target_addr]

    measurement_id = driver.ping(target_addr, vp_ids, tag)

end = timer()

print(f"number of targets: {nb_targets} : {end - start}")

## async measurements

In [16]:
import asyncio
import aiohttp
import time
import pickle

from random import choice
from timeit import default_timer as timer

from one_measurements.src.atlas_probing import RIPEAtlas

# load anchors
anchor_file = "../one_measurements/anchors_probes_dataset/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}


async def _wait_for(measurement_id, max_retry: int = 60):
    url = f"https://atlas.ripe.net/api/v2/measurements/{measurement_id}/results/?key={ripe_credentials['key']}"
    async with aiohttp.ClientSession(trust_env=True) as session:
        for _ in range(max_retry):
            async with session.get(url=url, ssl=False) as req:
                response = await req.json()

                if response:
                    return response
                time.sleep(2)


async def ping(target_addr, vp_ids, max_retry=60):
    url = f"https://atlas.ripe.net/api/v2/measurements/?key={ripe_credentials['key']}"
    json = {
        "definitions": [
            {
                "target": target_addr,
                "af": 4,
                "packets": 3,
                "size": 48,
                "description": f"Dioptra Geolocation of {target_addr}",
                "resolve_on_probe": False,
                "skip_dns_check": True,
                "include_probe_id": False,
                "type": "ping",
            }
        ],
        "probes": [
            {"value": vp, "type": "probes", "requested": 1}
            for vp in vp_ids
        ],
        "is_oneoff": True,
        "bill_to": ripe_credentials["username"],
    }

    async with aiohttp.ClientSession(trust_env=True) as session:
        for _ in range(max_retry):
            async with session.post(url=url, json=json, ssl=False) as req:
                print(req.status, target_addr)
                response = await req.json()

                try:
                    measurement_id = response["measurements"][0]
                    break
                except KeyError:
                    print(response)
                    print("Warning!", "Too much measurements.", "Waiting.")
                    time.sleep(60)

        results = await _wait_for(measurement_id)

        return {
            "target_addr": target_addr,
            "id": measurement_id,
            "results": results
        }


async def main(targets, vp_ids):
    measurements = await asyncio.gather(*[ping(target_addr, vp_ids) for target_addr in targets])
    return measurements

targets = [choice(list(anchors)) for _ in range(0, 5)]
vps = list(set(anchors).difference(set(targets)))[:5]
vp_ids = [anchors[vp_addr]['id'] for vp_addr in anchors]

start = timer()
# measurements = await main(targets, vp_ids)
end = timer()

In [17]:
print("nb target", len(targets), " vps per target:", len(vp_ids))
for measurement in measurements:
    print(f"{measurement['target_addr']} : {len(measurement['results'])}")

nb target 5  vps per target: 5


'for measurement in measurements:\n    print(f"{measurement[\'target_addr\']} : {len(measurement[\'results\'])}")'

In [ ]:
import pickle

from two_results_of_measurements.src.query_api import retreive_single_measurement

# load anchors
anchor_file = "../one_measurements/anchors_probes_dataset/anchors.pickle"
with open(anchor_file, "rb") as f:
    anchors = pickle.load(f)

ripe_credentials = {
    "username": "timur.friedman@sorbonne-universite.fr",
    "key": "b3d3d4fc-724e-4505-befe-1ad16a70dc87",
}

for measurement in measurements:
    results = retreive_single_measurement(
        measurement['id'], anchors,  ripe_credentials["key"])
    print(
        f"{measurement['target_addr']} : {len(measurement['results'])}, {len(results[measurement['target_addr']])}")